# Importing packages and files

In [ ]:
import json
import graphviz
import requests
from IPython.display import display
from anytree.exporter import DotExporter
import sys
sys.path.append('simulator/src')

URL = "127.0.0.1"
PORT = 8001

In [ ]:
with open("bpmn_test_api.json", "r") as f:
    bpmn_file = f.read()

bpmn = json.loads(bpmn_file)

# Initialize petri net and execution tree

In [ ]:
request_json = {
    "bpmn": bpmn
}

response = requests.post(f"http://{URL}:{PORT}/execute", headers={"Content-Type": "application/json"},
                         json=request_json)
response_json = response.json()

In [ ]:
bpmn = response_json['bpmn']
petri_net = response_json['petri_net']
petri_net_dot = response_json['petri_net_dot']
execution_tree = response_json['execution_tree']

In [ ]:
graph = graphviz.Source(petri_net_dot, format="svg")
display(graph)

# Execute choice

In [ ]:
request_json = {
    'bpmn': bpmn,
    'petri_net': petri_net,
    'petri_net_dot': petri_net_dot,
    'execution_tree': execution_tree,
    # 'choices': ['6'] # id delle transizioni con stop da eseguire, se non vengono passati vengono presi i default
}

response = requests.post(f"http://{URL}:{PORT}/execute", headers={"Content-Type": "application/json"},
                         json=request_json)
response_json = response.json()

In [ ]:
bpmn = response_json['bpmn']
petri_net = response_json['petri_net']
petri_net_dot = response_json['petri_net_dot']
execution_tree = response_json['execution_tree']

In [ ]:
graph = graphviz.Source(petri_net_dot, format="svg")
display(graph)

In [ ]:
from model.endpoints.execute.request import ExecuteRequest

model = ExecuteRequest.model_validate_json(response.text)
extree = model.execution_tree_obj
extree_dot = "\n".join(DotExporter(extree.root))
display(graphviz.Source(extree_dot))

# Display BPMN

In [ ]:
from paco.parser.dot import wrap_to_dot
from model.bpmn import get_active_region_by_pn

impacts_names = ["a", "b"]
marking = {"21": {"token": 1}, "7": {"token": 1}}

active_region_by_pn = get_active_region_by_pn(petri_net, marking)
dot = wrap_to_dot(bpmn, impacts_names, active_region_by_pn)
graph = graphviz.Source(dot, format="svg")

display(graph)

# Display Execution Tree

In [ ]:
from paco.parser.dot import get_path_to_current_node, wrapper_execution_tree_to_dot

current_path = get_path_to_current_node(execution_tree.get("root",{}), execution_tree.get("current_node"))
dot = wrapper_execution_tree_to_dot(execution_tree.get("root", {}), impacts_names, current_path)
graph = graphviz.Source(dot, format="svg")
display(graph)